# Object Pools

Let's start with a simple sketch, without worrying about type safety. All we really need, here, is a queue of objects. But when we pull objects from this pool, they need to come equipped with some means of rejoining the pool.

In [1]:
using DataStructures

In [2]:
mutable struct UnsafePool{T}
    q::Queue{T}
    UnsafePool{T}() where {T} = new(Queue{T}())
end

To do this, we'll wrap them in a struct that contains both the object and a reference back to the pool from whence it came.

In [3]:
mutable struct Recyclable{T}
    inner::Union{Nothing, T}
    home::Union{Nothing,Ref{UnsafePool{T}}}
end

In [4]:
function pull!(pool::UnsafePool{T})::Recyclable{T} where {T}
    item = dequeue!(pool.q)
    recyc = Recyclable{T}(item, Ref{UnsafePool{T}}(pool))
    return recyc
end

pull! (generic function with 1 method)

In [5]:
# for example
u = UnsafePool{Int}()
enqueue!(u.q, 1)
enqueue!(u.q, 2)
enqueue!(u.q, 3)
enqueue!(u.q, 4)
u

UnsafePool{Int64}(Queue{Int64}(Deque [[1, 2, 3, 4]]))

In [6]:
recyc = pull!(u)

Recyclable{Int64}(1, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))

In [9]:
function recycle(r::Recyclable{T}) where {T}
    if r.home === nothing
        error("Already recycled $r")
    end
    enqueue!(r.home[].q, r.inner)
    r.home = nothing
    r.inner = nothing
    return
end

recycle (generic function with 1 method)

In [10]:
# let's look at recyc, the wrapped object we pulled from the pool
println(recyc)

Recyclable{Int64}(1, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))


In [11]:
# and here's the pool it came from
println(u)

UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))


In [12]:
# maybe we do some work on recyc
recyc.inner += 1
println(recyc)

Recyclable{Int64}(2, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))


In [13]:
# now we're done with it, so we recycle it
println("recyc = $recyc")
println("pool before recycling: $u")
recycle(recyc)
println("pool after recycling: $u")



recyc = Recyclable{Int64}(2, Base.RefValue{UnsafePool{Int64}}(UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))))
pool before recycling: UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4]]))
pool after recycling: UnsafePool{Int64}(Queue{Int64}(Deque [[2, 3, 4, 2]]))


In [14]:
println("recyc after recycling (throw this away): $recyc")

recyc after recycling (throw this away): Recyclable{Int64}(nothing, nothing)
